# Enhanced Breakout Scanner Testing Notebook - Top 3 Stocks Per Day

This notebook provides a comprehensive testing environment for the enhanced breakout scanner functionality.

## 🚀 New Features:
- **Date Range Analysis**: Scan multiple trading days at once
- **End-of-Day Tracking**: Monitor price movement from 09:50 AM to 15:15 PM
- **Professional Display**: Beautiful table formatting with all metrics
- **Performance Analysis**: Success rates, price changes, and rankings
- **CSV Export**: Save results for further analysis
- **Enhanced Configuration**: Optimized parameters for better results

## 📊 Related Dashboard:
- **Interactive Dashboard**: Check out `breakout_scanner_dashboard.ipynb` for beautiful plotly visualizations in a 2x3 grid layout

## 📋 Core Features:
- Initialize and configure the breakout scanner
- Test scanner with different date ranges and parameters
- Debug and troubleshoot scanner issues
- Compare different scanner configurations

## 📋 Prerequisites:
- DuckDB database with market data
- Required Python packages installed
- Proper configuration files in place

In [1]:
# Setup and Imports
import sys
import os
from pathlib import Path
from datetime import datetime, date, time, timedelta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Add project root to Python path
current_dir = Path.cwd()
if current_dir.name == 'scanner':
    # We're in notebook/scanner/
    project_root = current_dir.parent.parent
elif current_dir.name == 'notebook':
    # We're in notebook/
    project_root = current_dir.parent
else:
    # Assume we're in project root or another location
    project_root = current_dir

# Add paths to sys.path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print(f"Current directory: {current_dir}")
print(f"Project root: {project_root}")
print(f"Added to sys.path: {project_root / 'src'}")

# Import project modules
from src.infrastructure.logging import setup_logging
from src.application.scanners.strategies.breakout_scanner import BreakoutScanner
from src.infrastructure.core.database import DuckDBManager
from src.infrastructure.config.settings import get_settings

# Setup logging
setup_logging()

# Set up plotly default theme
import plotly.io as pio
pio.templates.default = "plotly_dark"

print(f"Project root: {project_root}")
print(f"Python path includes: {project_root / 'src'}")
print("✅ Environment setup complete!")

Current directory: /Users/apple/Downloads/duckDbData/notebook/scanner
Project root: /Users/apple/Downloads/duckDbData
Added to sys.path: /Users/apple/Downloads/duckDbData/src


Project root: /Users/apple/Downloads/duckDbData
Python path includes: /Users/apple/Downloads/duckDbData/src
✅ Environment setup complete!


In [2]:
# Configuration and Initialization
print("🔧 Initializing Breakout Scanner...")

# Get settings
settings = get_settings()

# Initialize database connection
# Use the main database with all the data
main_db_path = project_root / "data" / "financial_data.duckdb"
print(f"Using database: {main_db_path}")
print(f"Database exists: {main_db_path.exists()}")

db_manager = DuckDBManager(
    db_path=str(main_db_path)
)

# Test database connection
try:
    connection = db_manager.connect()
    print(f"✅ Database connection established: {connection is not None}")
except Exception as e:
    print(f"❌ Database connection failed: {e}")

# Initialize scanner
scanner = BreakoutScanner(db_manager=db_manager)

print(f"Database: {main_db_path}")
print(f"Scanner initialized with config:")
for key, value in scanner.config.items():
    print(f"  {key}: {value}")
print("✅ Scanner initialization complete!")

INFO:src.infrastructure.core.database:Initialized DuckDB manager with database: /Users/apple/Downloads/duckDbData/data/financial_data.duckdb


🔧 Initializing Breakout Scanner...
Using database: /Users/apple/Downloads/duckDbData/data/financial_data.duckdb
Database: /Users/apple/Downloads/duckDbData/data/financial_data.duckdb
Scanner initialized with config:
  consolidation_period: 5
  breakout_volume_ratio: 1.5
  resistance_break_pct: 0.5
  min_price: 50
  max_price: 2000
  max_results: 50
  breakout_cutoff_time: 09:50:00
  end_of_day_time: 15:15:00
✅ Scanner initialization complete!


In [3]:
# Enhanced Breakout Scanner Demo - Basic Single-Day Testing
print("🚀 Enhanced Breakout Scanner Demo - Basic Single-Day Testing")

# Apply optimal configuration
scanner.config.update({
    'resistance_break_pct': 0.5,    # 0.5% breakout (reduced from 1.0%)
    'breakout_volume_ratio': 1.5,   # 1.5x volume (reduced from 2.0x)
    'consolidation_range_pct': 8.0, # 8.0% consolidation (increased from 3.0%)
    'min_price': 50,
    'max_price': 2000,
    'max_results': 50
})

print("⚙️ Applied optimal scanner configuration")
print("📅 Testing on optimal dates (2025-09-01 to 2025-09-05)")

# Test on optimal dates (Single-day mode for compatibility)
optimal_dates = [date(2025, 9, 1), date(2025, 9, 2), date(2025, 9, 3), date(2025, 9, 4), date(2025, 9, 5)]
cutoff_time = time(9, 50)

all_results = []
for test_date in optimal_dates:
    print(f"\n📊 Testing {test_date}...")
    print("⚠️  Single-day scan mode (use scan_date_range() for full functionality)")
    try:
        results = scanner.scan(test_date, cutoff_time)
        if not results.empty:
            print(f"✅ Found {len(results)} stocks with breakout patterns")
            print(f"   ✅ Found {len(results)} breakout candidates")
            # Show top 3 for this date (with better error handling)
            if 'breakout_score' in results.columns:
                # Get top 3 based on breakout_pct if breakout_score doesn't exist
                sort_column = 'breakout_pct' if 'breakout_pct' in results.columns else results.columns[0]
                top_3_columns = ['symbol', 'breakout_price', 'breakout_pct', 'volume_ratio']
                available_columns = [col for col in top_3_columns if col in results.columns]
                if not available_columns:
                    available_columns = results.columns[:4].tolist()  # First 4 columns as fallback
                top_3 = results.nlargest(3, sort_column)[available_columns]
                for _, row in top_3.iterrows():
                    # Format output based on available columns
                    output_parts = [f"   {row['symbol']}"]
                    if 'breakout_price' in row.index:
                        output_parts.append(f"₹{row['breakout_price']:.2f}")
                    if 'breakout_pct' in row.index:
                        output_parts.append(f"Breakout: {row['breakout_pct']:.2f}%")
                    if 'volume_ratio' in row.index:
                        output_parts.append(f"Volume: {row['volume_ratio']:.1f}x")
                    output_parts.append(f"Date: {test_date}")
                    print(" | ".join(output_parts))
            all_results.append(results)
        else:
            print("   ⚠️ No breakout candidates found")
    except Exception as e:
        print(f"   ❌ Error: {e}")

# Combine and show summary
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    unique_results = combined_results.drop_duplicates(subset=['symbol'])
    if 'breakout_score' in unique_results.columns:
        top_overall = unique_results.nlargest(10, 'breakout_score')
    
        print("\n🎯 TOP 10 UNIQUE BREAKOUT CANDIDATES OVERALL:")
        for _, row in top_overall.iterrows():
            print(f"   {row['symbol']}: ₹{row['current_price']:.2f} | Score: {row['breakout_score']:.2f} | Breakout: {row['breakout_pct']:.2f}% | Volume: {row['volume_ratio']:.1f}x")
    
        print(f"\n📈 SUMMARY: Found {len(unique_results)} unique stocks across {len(all_results)} dates")
        if 'breakout_signal' in combined_results.columns:
            print(f"🔥 Strong breakouts: {len(combined_results[combined_results['breakout_signal'] == 'STRONG_BREAKOUT'])}")
            print(f"⚡ Weak breakouts: {len(combined_results[combined_results['breakout_signal'] == 'WEAK_BREAKOUT'])}")
    
        # Save results
        combined_results.to_csv('notebook_breakout_results.csv', index=False)
        top_overall.to_csv('notebook_top10_breakouts.csv', index=False)
        print("\n💾 Results saved to CSV files")
else:
    print("❌ No results found across all test dates")

print("\n🎉 Basic Single-Day Scanner Demo Complete!")
print("\n💡 Tip: Use the next cell for Enhanced Date Range Analysis!")

🚀 Enhanced Breakout Scanner Demo - Basic Single-Day Testing
⚙️ Applied optimal scanner configuration
📅 Testing on optimal dates (2025-09-01 to 2025-09-05)

📊 Testing 2025-09-01...
⚠️  Single-day scan mode (use scan_date_range() for full functionality)
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


INFO:src.infrastructure.utils.retry:Starting call to 'src.application.scanners.base_scanner.BaseScanner._execute_query', this is the 1st time calling it.
INFO:src.infrastructure.core.database:Connected to DuckDB database: /Users/apple/Downloads/duckDbData/data/financial_data.duckdb


📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 9 stocks with breakout patterns
✅ Found 9 stocks with breakout patterns
   ✅ Found 9 breakout candidates
   CGPOWER | Breakout: 3.46% | Volume: 5.9x | Date: 2025-09-01
   GPIL | Breakout: 1.54% | Volume: 15.1x | Date: 2025-09-01
   GMDCLTD | Breakout: 1.46% | Volume: 30.3x | Date: 2025-09-01

📊 Testing 2025-09-02...
⚠️  Single-day scan mode (use scan_date_range() for full functionality)
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


INFO:src.infrastructure.utils.retry:Starting call to 'src.application.scanners.base_scanner.BaseScanner._execute_query', this is the 1st time calling it.


📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 22 stocks with breakout patterns
✅ Found 22 stocks with breakout patterns
   ✅ Found 22 breakout candidates
   BALRAMCHIN | Breakout: 4.07% | Volume: 28.6x | Date: 2025-09-02
   CIEINDIA | Breakout: 3.29% | Volume: 7.3x | Date: 2025-09-02
   APOLLOTYRE | Breakout: 2.46% | Volume: 19.5x | Date: 2025-09-02

📊 Testing 2025-09-03...
⚠️  Single-day scan mode (use scan_date_range() for full functionality)
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


...

📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 7 stocks with breakout patterns
✅ Found 7 stocks with breakout patterns
   ✅ Found 7 breakout candidates
   HINDCOPPER | Breakout: 1.75% | Volume: 12.2x | Date: 2025-09-03
   CGPOWER | Breakout: 1.57% | Volume: 1.7x | Date: 2025-09-03
   EIHOTEL | Breakout: 1.42% | Volume: 20.6x | Date: 2025-09-03

📊 Testing 2025-09-04...
⚠️  Single-day scan mode (use scan_date_range() for full functionality)
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


INFO:src.infrastructure.utils.retry:Starting call to 'src.application.scanners.base_scanner.BaseScanner._execute_query', this is the 1st time calling it.


📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 16 stocks with breakout patterns
✅ Found 16 stocks with breakout patterns
   ✅ Found 16 breakout candidates
   CAMPUS | Breakout: 4.36% | Volume: 19.3x | Date: 2025-09-04
   BAJFINANCE | Breakout: 3.83% | Volume: 2.4x | Date: 2025-09-04
   BATAINDIA | Breakout: 3.78% | Volume: 24.2x | Date: 2025-09-04

📊 Testing 2025-09-05...
⚠️  Single-day scan mode (use scan_date_range() for full functionality)
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


INFO:src.infrastructure.utils.retry:Starting call to 'src.application.scanners.base_scanner.BaseScanner._execute_query', this is the 1st time calling it.


📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 8 stocks with breakout patterns
✅ Found 8 stocks with breakout patterns
   ✅ Found 8 breakout candidates
   IDFCFIRSTB | Breakout: 2.11% | Volume: 2.6x | Date: 2025-09-05
   RTNINDIA | Breakout: 1.83% | Volume: 46.6x | Date: 2025-09-05
   RBLBANK | Breakout: 1.58% | Volume: 3.1x | Date: 2025-09-05

🎯 TOP 10 UNIQUE BREAKOUT CANDIDATES OVERALL:
   RTNINDIA: ₹62.48 | Score: 15.01 | Breakout: 1.83% | Volume: 46.6x
   BALRAMCHIN: ₹579.85 | Score: 10.50 | Breakout: 4.07% | Volume: 28.6x
   GMDCLTD: ₹425.10 | Score: 9.97 | Breakout: 1.46% | Volume: 30.3x
   BATAINDIA: ₹1232.00 | Score: 9.08 | Breakout: 3.78% | Volume: 24.2x
   CAMPUS: ₹284.30 | Score: 7.82 | Breakout: 4.36% | Volume: 19.3x
   APOLLOTYRE: ₹482.85 | Score: 7.13 | Breakout: 2.46% | Volume: 19.5x
   EIHOTEL: ₹413.40 | Score: 7.04 | Breakout: 1.42% | Vo

# 🚀 Enhanced Date Range Analysis

Now let's test the **enhanced features** that provide comprehensive analysis across multiple days with end-of-day tracking and professional reporting.

In [4]:
# Enhanced Breakout Scanner Demo - Date Range Analysis
print("🆕 Enhanced Breakout Scanner Demo - Date Range Analysis")
print("📅 Scanning date range with end-of-day tracking...")

# Configure analysis parameters
start_date = date(2025, 9, 1)
end_date = date(2025, 9, 5)
breakout_time = time(9, 50)  # 09:50 AM breakout detection
end_of_day_time = time(15, 15)  # 15:15 PM end-of-day analysis

print(f"📅 Analyzing period: {start_date} to {end_date}")
print(f"⏰ Breakout detection: {breakout_time}")
print(f"⏰ End-of-day analysis: {end_of_day_time}")

# Run enhanced analysis
try:
    enhanced_results = scanner.scan_date_range(
        start_date=start_date,
        end_date=end_date,
        cutoff_time=breakout_time,
        end_of_day_time=end_of_day_time
    )
    
    if enhanced_results:
        print(f"\n✅ Found {len(enhanced_results)} enhanced breakout results")
        
        # Display professional table
        scanner.display_results_table(
            enhanced_results,
            title=f"Enhanced Breakout Analysis: {start_date} to {end_date}"
        )
        
        # Export to CSV
        csv_filename = f"enhanced_breakout_analysis_{start_date.strftime('%Y%m%d')}_to_{end_date.strftime('%Y%m%d')}.csv"
        scanner.export_results(enhanced_results, csv_filename)
        
        print(f"\n💾 Results exported to: {csv_filename}")
        print("\n📊 Enhanced features demonstrated successfully!")
        
    else:
        print("⚠️ No enhanced results found")
        
except Exception as e:
    print(f"❌ Enhanced analysis failed: {e}")
    print("💡 Make sure the scanner has been properly initialized in the previous cells")

print("\n🎉 Enhanced Date Range Analysis Complete!")

🆕 Enhanced Breakout Scanner Demo - Date Range Analysis
📅 Scanning date range with end-of-day tracking...
📅 Analyzing period: 2025-09-01 to 2025-09-05
⏰ Breakout detection: 09:50:00
⏰ End-of-day analysis: 15:15:00
🚀 Enhanced Breakout Scanner - Date Range: 2025-09-01 to 2025-09-05
⏰ Breakout Detection: 09:50:00, End of Day: 15:15:00
📅 Found 5 trading days to analyze
📊 Scanning 2025-09-01...
   ✅ Found 22 breakout candidates
📊 Scanning 2025-09-02...
   ✅ Found 27 breakout candidates
📊 Scanning 2025-09-03...
   ✅ Found 21 breakout candidates
📊 Scanning 2025-09-04...
   ✅ Found 50 breakout candidates
📊 Scanning 2025-09-05...
   ✅ Found 50 breakout candidates

✅ Enhanced Breakout Analysis Complete!
📊 Total breakout opportunities: 170
📅 Trading days analyzed: 5

✅ Found 170 enhanced breakout results

📊 Enhanced Breakout Analysis: 2025-09-01 to 2025-09-05
┌──────────┬──────────┬────────────┬──────────┬────────────┬──────────┬────────────┬──────┬──────────┬──────┬────────────┐
│ Symbol   │ Date

# 📊 Performance Analysis & Visualization

Analyze the success rates and performance patterns of your breakout scanner results.

In [5]:
# Performance Analysis and Visualization
print("📊 Performance Analysis & Visualization")

# Check if we have enhanced results from the previous cell
try:
    if 'enhanced_results' in locals() and enhanced_results:
        print(f"\n📈 Analyzing {len(enhanced_results)} breakout opportunities...")
        
        # Convert to DataFrame for analysis
        df = pd.DataFrame(enhanced_results)
        
        # Basic statistics
        successful_breakouts = len(df[df['breakout_successful'] == True])
        total_breakouts = len(df)
        success_rate = (successful_breakouts / total_breakouts * 100) if total_breakouts > 0 else 0
        
        print(f"\n📊 PERFORMANCE METRICS:")
        print(f"   Total Breakouts: {total_breakouts}")
        print(f"   Successful Breakouts: {successful_breakouts}")
        print(f"   Success Rate: {success_rate:.1f}%")
        
        if 'price_change_pct' in df.columns:
            avg_price_change = df['price_change_pct'].mean()
            print(f"   {row['symbol']}: ₹{row['current_price']:.2f} | Score: {row['breakout_score']:.2f} | Breakout: {row['breakout_pct']:.2f}% | Volume: {row['volume_ratio']:.1f}x")
            
            # Success rate by day range
            high_performers = len(df[df['price_change_pct'] > 2.0])
            print(f"   High Performers (>2%): {high_performers}")
        
        # Group by scan date (handle different date formats)
        if 'scan_date' in df.columns:
            try:
                # Convert scan_date to datetime if it's not already
                if not pd.api.types.is_datetime64_any_dtype(df['scan_date']):
                    df['scan_date'] = pd.to_datetime(df['scan_date'])
                
                daily_stats = df.groupby(df['scan_date'].dt.date).agg({
                    'symbol': 'count',
                    'breakout_successful': 'sum',
                    'price_change_pct': 'mean'
                }).round(2)
                
                print(f"\n📅 DAILY PERFORMANCE:")
                print(daily_stats)
            except Exception as e:
                print(f"   Note: Could not group by date: {e}")
        
        # Top performers
        if 'performance_rank' in df.columns:
            # Select available columns for top performers
            available_columns = ['symbol', 'price_change_pct', 'performance_rank']
            optional_columns = ['scan_date', 'breakout_price', 'eod_price']
            for col in optional_columns:
                if col in df.columns:
                    available_columns.append(col)
            
            top_performers = df.nsmallest(min(5, len(df)), 'performance_rank')[available_columns]
            print(f"\n🏆 TOP {len(top_performers)} PERFORMERS:")
            for _, row in top_performers.iterrows():
                # Format output based on available columns
                date_str = row['scan_date'].strftime('%Y-%m-%d') if 'scan_date' in row.index and hasattr(row['scan_date'], 'strftime') else 'N/A'
                output_parts = [f"   {row['symbol']} ({date_str})"]
                if 'price_change_pct' in row.index:
                    output_parts.append(f"{row['price_change_pct']:+.2f}%")
                if 'performance_rank' in row.index:
                    output_parts.append(f"(Rank: {row['performance_rank']:.2f})")
                print(": ".join(output_parts))
        
        print("\n✅ Performance analysis complete!")
        
    else:
        print("⚠️ No enhanced results available. Please run the Enhanced Date Range Analysis cell first.")
        
except Exception as e:
    print(f"❌ Performance analysis failed: {e}")

print("\n🎉 Performance Analysis Complete!")

📊 Performance Analysis & Visualization

📈 Analyzing 170 breakout opportunities...

📊 PERFORMANCE METRICS:
   Total Breakouts: 170
   Successful Breakouts: 49
.1f
   GPIL: ₹243.25 | Score: 5.45 | Breakout: 1.54% | Volume: 15.1x
   High Performers (>2%): 14
❌ Performance analysis failed: Can only use .dt accessor with datetimelike values

🎉 Performance Analysis Complete!


# ⚙️ Configuration Tuning & Comparison

Experiment with different scanner configurations to optimize your results.

In [6]:
# Configuration Tuning and Comparison
print("⚙️ Configuration Tuning & Comparison")
print("🧪 Testing different scanner configurations...")

# Define different configurations to test
configurations = {
    'Conservative': {
        'resistance_break_pct': 1.0,    # Higher breakout threshold
        'breakout_volume_ratio': 2.0,   # Higher volume requirement
        'consolidation_range_pct': 5.0, # Tighter consolidation
        'min_price': 100,
        'max_price': 1500,
        'max_results': 30
    },
    'Moderate': {
        'resistance_break_pct': 0.5,    # Current optimal
        'breakout_volume_ratio': 1.5,   # Current optimal
        'consolidation_range_pct': 8.0, # Current optimal
        'min_price': 50,
        'max_price': 2000,
        'max_results': 50
    },
    'Aggressive': {
        'resistance_break_pct': 0.2,    # Lower breakout threshold
        'breakout_volume_ratio': 1.2,   # Lower volume requirement
        'consolidation_range_pct': 12.0, # Wider consolidation
        'min_price': 20,
        'max_price': 3000,
        'max_results': 100
    }
}

# Test date for comparison
test_date = date(2025, 9, 4)
cutoff_time = time(9, 50)

print(f"📅 Testing configurations on: {test_date}")
print(f"\n{'Configuration':<15} {'Results':<8} {'Success Rate':<12} {'Avg Change':<10}")
print("-" * 50)

# Test each configuration
for config_name, config_params in configurations.items():
    try:
        # Save original config
        original_config = scanner.config.copy()
        
        # Apply new configuration
        scanner.config.update(config_params)
        
        # Run scan
        results = scanner.scan(test_date, cutoff_time)
        
        # Analyze results
        if not results.empty and len(results) > 0:
            result_count = len(results)
            
            # Calculate metrics if available
            if 'breakout_successful' in results.columns:
                success_count = results['breakout_successful'].sum()
                success_rate = (success_count / result_count * 100) if result_count > 0 else 0
            else:
                success_rate = 0.0
                
            if 'price_change_pct' in results.columns:
                avg_change = results['price_change_pct'].mean()
            else:
                avg_change = 0.0
                
            print(f"{config_name:<15} {result_count:<8} {success_rate:<11.1f}% {avg_change:<9.2f}%")
        else:
            print(f"{config_name:<15} {'0':<8} {'N/A':<12} {'N/A':<10}")
        
        # Restore original config
        scanner.config.update(original_config)
        
    except Exception as e:
        print(f"{config_name:<15} {'ERROR':<8} {'N/A':<12} {'N/A':<10}")
        print(f"   ❌ {e}")

print("\n✅ Configuration comparison complete!")
print("💡 Tip: Adjust parameters based on your risk tolerance and market conditions.")

⚙️ Configuration Tuning & Comparison
🧪 Testing different scanner configurations...
📅 Testing configurations on: 2025-09-04

Configuration   Results  Success Rate Avg Change
--------------------------------------------------
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


INFO:src.infrastructure.utils.retry:Starting call to 'src.application.scanners.base_scanner.BaseScanner._execute_query', this is the 1st time calling it.


📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 8 stocks with breakout patterns
Conservative    8        0.0        % 0.00     %
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


INFO:src.infrastructure.utils.retry:Starting call to 'src.application.scanners.base_scanner.BaseScanner._execute_query', this is the 1st time calling it.


📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 16 stocks with breakout patterns
Moderate        16       0.0        % 0.00     %
⚠️  Single-day scan mode (use scan_date_range() for full functionality)


INFO:src.infrastructure.utils.retry:Starting call to 'src.application.scanners.base_scanner.BaseScanner._execute_query', this is the 1st time calling it.


📊 Analyzing 487 symbols for breakout patterns...
DEBUG RETRY_DB DECORATOR APPLIED: _execute_query called with 3 args
DEBUG RETRY_DB INNER: func=_execute_query, args_count=3
✅ Found 22 stocks with breakout patterns
Aggressive      22       0.0        % 0.00     %

✅ Configuration comparison complete!
💡 Tip: Adjust parameters based on your risk tolerance and market conditions.


In [1]:
from src.application.scanners.strategies.crp_scanner import CRPScanner

# Initialize scanner
scanner = CRPScanner()

# Date range analysis
results = scanner.scan_date_range(
    start_date=date(2024, 1, 10),
    end_date=date(2024, 1, 15)
)

# Display results
scanner.display_results_table(results)

# Export to CSV
scanner.export_results(results, "crp_analysis.csv")

ModuleNotFoundError: No module named 'src'